In [4]:
import json
import common
# Get a configured model
client = common.get_openai_client()

In [6]:
# Defined intents
RAG_INTENT="AnswerQuestion"
QUERY_INTENT="QueryDatabase"

# Initialize the intent Prompt template
intent_template = """An intent is an action from the list of defined intents.

Defined intents:
AnswerQuestion
QueryDatabase


What is the intent for the following request:
{{input}}

Output format:
{
  "intent": "intent",
  "tables": [""]
}

Do not provide explanations. Output in JSON format only.
"""

# Validate the JSON returned from the intent Prompt
def validate_json(payload:str) -> bool:
    check = json.loads(payload)
    # the JSON must have an intent field and should be either AnswerQuestion or QueryDatabase
    if "intent" in check and check["intent"] in [RAG_INTENT,QUERY_INTENT]:
        return True
    else:
        return False

# Get the intent
def get_intent(payload:str) -> str:
    check = json.loads(payload)
    return check["intent"]

# If it is a query intent, get the tables participating in the query
def get_tables_from_json(payload:str) -> list[str]:
    check = json.loads(payload)
    return check["tables"]

# Mock a RAG call
def mock_rag():
    return """Water is an inorganic compound with the chemical formula H2O. It is a transparent, tasteless, odorless,[c] and nearly colorless chemical substance, and it is the main constituent of Earth's hydrosphere and the fluids of all known living organisms (in which it acts as a solvent[20]). It is vital for all known forms of life, despite not providing food energy or organic micronutrients. Its chemical formula, H2O, indicates that each of its molecules contains one oxygen and two hydrogen atoms, connected by covalent bonds. The hydrogen atoms are attached to the oxygen atom at an angle of 104.45°.[21] "Water" is also the name of the liquid state of H2O at standard temperature and pressure.\nSource: wikipedia.org\n\n"""

# Mock getting a table schema
def mock_get_table_schema(table):
    if table == "customer":
      return """The customer table has the following fields: customer_id, customer_name"""
    elif table == "sales":
       return """The sales table has the following fields: order_id, customer_id, order_date, order_amount, amount"""
    else:
        return ""

# Process the intent
def process_for_intent(intentJson:str,input:str) -> str:
  if validate_json(intentJson):
      intent = get_intent(intentJson)      
      if intent == RAG_INTENT:
          rag_template = input + "\n" + mock_rag() +"\n\nUse only the provided text. Quote the sources."
          answer = common.Call_OpenAI(client,common.gpt_api_deployment,rag_template, max_tokens=500, temperature=0.2)
          print(f'A: {common.format_output(answer)}')
      elif intent == QUERY_INTENT:
          tables = get_tables_from_json(intentJson)
          query_template = input +"\n\n"
          for table in tables:
            query_template += mock_get_table_schema(table) + "\n"
          query_template += "\n\nUse only the provided text."
          #print(f'Query: {query_template}')          
          answer = common.Call_OpenAI(client,common.gpt_api_deployment,query_template, max_tokens=200, temperature=0.2)          
          print(f'A: {common.format_output(answer)}')
      else:
          print("Intent not supported")

## Process questions with different intents

In [7]:
# Ask the first question
input = "What is the chemical composition of water?"
# Render the template
template = common.render_template(intent_template,input=input)
# Send the completed Prompt to OpenAI for Completion
intent = common.Call_OpenAI(client,common.gpt_api_deployment,template)

# Print the question and the processed intent
print(f'Q: {common.format_output(input)}')
process_for_intent(intent,input)

# Ask the second question
input = "Write a SQL query to get the customer id and name with the highest balance from the customer table joined to the sales table by customer id."
# Render the template
template = common.render_template(intent_template,input=input)
# Send the completed Prompt to OpenAI for Completion
intent = common.Call_OpenAI(client,common.gpt_api_deployment,template)

# Print the question and the processed intent
print(f'\nQ: {common.format_output(input)}')
process_for_intent(intent,input)

Q: What is the chemical composition of water?

A: "Water is an inorganic compound with the chemical formula H2O. It is a
transparent, tasteless, odorless, and nearly colorless chemical substance, and
it is the main constituent of Earth's hydrosphere and the fluids of all known
living organisms (in which it acts as a solvent). It is vital for all known
forms of life, despite not providing food energy or organic micronutrients. Its
chemical formula, H2O, indicates that each of its molecules contains one oxygen
and two hydrogen atoms, connected by covalent bonds. The hydrogen atoms are
attached to the oxygen atom at an angle of 104.45°. "Water" is also the name of
the liquid state of H2O at standard temperature and pressure." (Source:
wikipedia.org)


Q: Write a SQL query to get the customer id and name with the highest balance from
the customer table joined to the sales table by customer id.

A: SELECT c.customer_id, c.customer_name FROM customer c JOIN sales s ON
c.customer_id = s.custo